In [1]:
! pip install transformers

import numpy as np
import pandas as pd

import torch
import sys
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler#, get_linear_schedule_with_warmup
#from pytorch_pretrained_bert import BertTokenizer, BertConfig
from sklearn.metrics import recall_score, precision_score
from tqdm import trange


from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup, AdamW

     |████████████████████████████████| 2.6 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 636 kB 33.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r -f /content/drive/'My Drive'/msu/re_from_bioBert /content/
import sys

sys.path.append('/content/msu/re_from_bioBert/')
dir_name = '/content/re_from_bioBert/'

Mounted at /content/drive


In [3]:
def fit(model, x, y, n_epochs, lr, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]
    
    t_inputs = torch.tensor(x)
    t_labels = torch.tensor(y)
    t_masks = torch.tensor(masks)

    train_data = TensorDataset(t_inputs, t_masks, t_labels)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.cuda()

    param_optimizer = list(model.named_parameters())
    no_decay  = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    
    optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=lr,
    eps=1e-6)

    epochs = n_epochs
    max_grad_norm = 1.5

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.


    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0.1 * total_steps,
        num_training_steps=total_steps)
    
    model.train()
    for _ in trange(epochs, desc="Epoch"):
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            model.zero_grad()
        
            outputs = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
        
            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()

            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            optimizer.step()
            scheduler.step()
        print(' Loss: ', total_loss)    


def predict(model, x, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]

    t_inputs = torch.tensor(x)
    t_masks = torch.tensor(masks)

    test_data = TensorDataset(t_inputs, t_masks)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions = []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        outputs = outputs[0].detach().cpu().numpy()
        
        batch_labels = np.argmax(outputs, axis=1).tolist()
        predictions += batch_labels
    return predictions

def get_data_from_df(df, tokenizer):
    lbls, raw_txt = df.lbl.values, df.txt.values
    txt = ['[CLS] ' + i + ' [SEP]' for i in raw_txt]
    res = []
    for sent in txt:
      tokens = tokenizer.tokenize(sent)
      tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
      while len(tokens_ids) < MAX_LEN:
        tokens_ids.append(0)
      res.append(tokens_ids)
    return res, lbls
    

def cv_iteration(model, tokenizer, data_dir, cur_it, n_epochs, lr, batch_size=32):
    train_df = pd.read_csv(data_dir + '/'+ str(cur_it+1) + '/train.tsv', sep = '\t', names = ['txt', 'lbl'])
    train_tokens, train_lbls = get_data_from_df(train_df, tokenizer)
    train_lbls = [int(i) for i in train_lbls]
    
    test_df = pd.read_csv(data_dir + '/'+ str(cur_it+1) + '/test.tsv', sep = '\t', names = ['txt', 'lbl'])
    test_tokens, test_lbls = get_data_from_df(test_df, tokenizer)
    test_tokens, test_lbls = test_tokens[1:], test_lbls[1:]
    test_lbls = [int(i) for i in test_lbls]
    
    
    fit(model, train_tokens, train_lbls, n_epochs, lr, batch_size)
    preds = predict(model, test_tokens, batch_size)
    #print(np.bincount(np.array(train_tokens)), np.bincount(np.array(preds)))

    precision, recall = precision_score(test_lbls, preds), recall_score(test_lbls, preds)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1


def get_cv_scores(model_name, tokenizer, data_dir, lr,  n_epochs, batch_size=32, st_from=0):
    prescision_s, recall_s, f1_s = [], [], []
    for i in range(st_from, 10):
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        p, r, f = cv_iteration(model, tokenizer, data_dir, i, n_epochs, lr, batch_size)
        print('P: ', p, 'R: ', r, 'F1: ', f)
        prescision_s.append(p)
        recall_s.append(r)
        f1_s.append(f)
    return sum(prescision_s) / len(prescision_s), sum(recall_s) / len(recall_s), sum(f1_s) / len(f1_s) 


In [ ]:
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = "dmis-lab/biobert-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 3, 32)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:  33%|███▎      | 1/3 [02:02<04:04, 122.36s/it]

 Loss:  91.39673164486885


Epoch:  67%|██████▋   | 2/3 [04:09<02:03, 123.77s/it]

 Loss:  68.3507828116417


Epoch: 100%|██████████| 3/3 [06:17<00:00, 125.67s/it]

...

Epoch:  67%|██████▋   | 2/3 [04:15<02:07, 127.59s/it]

 Loss:  66.50431564450264


Epoch: 100%|██████████| 3/3 [06:22<00:00, 127.46s/it]

 Loss:  52.94078643620014


P:  0.7155172413793104 R:  0.8892857142857142 F1:  0.7929936305732483
P:  0.7499030628844078 R:  0.8943124046771734 F1:  0.8153220493148933


In [4]:
# st
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = "dmis-lab/biobert-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 5, 16)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

...

Epoch:  80%|████████  | 4/5 [05:27<01:21, 81.94s/it]

 Loss:  70.54833816550672


Epoch: 100%|██████████| 5/5 [06:49<00:00, 81.94s/it]

 Loss:  53.16209271363914


P:  0.75 R:  0.8571428571428571 F1:  0.7999999999999999
P:  0.772587610611932 R:  0.8636082867310625 F1:  0.81530631979596


In [ ]:
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
model = "dmis-lab/biobert-large-cased-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 5, 16)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

 Loss:  176.25494734942913


Epoch:  40%|████      | 2/5 [07:55<11:53, 237.93s/it]

 Loss:  129.74050809070468


Epoch:  60%|██████    | 3/5 [11:53<07:55, 237.90s/it]

...

Epoch:  80%|████████  | 4/5 [15:55<03:58, 238.88s/it]

 Loss:  209.78010261058807


Epoch: 100%|██████████| 5/5 [19:54<00:00, 238.88s/it]

 Loss:  208.9656002521515


P:  0.5263157894736842 R:  1.0 F1:  0.6896551724137931
P:  0.7297504232962622 R:  0.8746886120996441 F1:  0.7858268197053467


In [ ]:
MAX_LEN = 150

model = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model)
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 5, 16)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

 Loss:  177.69122552871704


Epoch:  40%|████      | 2/5 [02:42<04:03, 81.18s/it]

 Loss:  125.12045377492905


Epoch:  60%|██████    | 3/5 [04:03<02:42, 81.18s/it]

...

Epoch:  80%|████████  | 4/5 [05:24<01:21, 81.19s/it]

 Loss:  74.10094687342644


Epoch: 100%|██████████| 5/5 [06:46<00:00, 81.21s/it]

 Loss:  59.3399417065084


P:  0.7360703812316716 R:  0.8964285714285715 F1:  0.8083735909822866
P:  0.7658913858831704 R:  0.8814806812404676 F1:  0.8192921054859447
